In [1]:
%matplotlib inline
import pandas as pd

In [83]:
df = pd.read_csv('CollegeScorecard_Raw_Data/MERGED2015_16_PP.csv')

/home/chris/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,9,1725,1726,1727,1728,1729,1743) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [84]:
df.head()

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,D150_L4_WHITENH,D150_L4_BLACKNH,D150_L4_API,D150_L4_AIANOLD,D150_L4_HISPOLD,D_PCTPELL_PCTFLOAN,OPENADMP,UGNONDS,GRADS,ACCREDCODE
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,www2.aamu.edu/scripts/netpricecalc/npcalc.htm,...,NaN,NaN,NaN,NaN,NaN,4210.0,2.0,NaN,1123.0,SACSCC
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,www.uab.edu,uab.studentaidcalculator.com/survey.aspx,...,NaN,NaN,NaN,NaN,NaN,11679.0,2.0,242.0,6822.0,SACSCC
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,www.amridgeuniversity.edu,www2.amridgeuniversity.edu:9091/,...,NaN,NaN,NaN,NaN,NaN,279.0,1.0,NaN,319.0,SACSCC
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu,finaid.uah.edu/,...,NaN,NaN,NaN,NaN,NaN,5618.0,2.0,184.0,1853.0,SACSCC
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu,www.alasu.edu/cost-aid/forms/calculator/index....,...,NaN,NaN,NaN,NaN,NaN,4805.0,2.0,24.0,619.0,SACSCC


In [85]:
yale = df[df.INSTURL.fillna('').map(lambda val: 'yale.edu' in val)]
yale

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,D150_L4_WHITENH,D150_L4_BLACKNH,D150_L4_API,D150_L4_AIANOLD,D150_L4_HISPOLD,D_PCTPELL_PCTFLOAN,OPENADMP,UGNONDS,GRADS,ACCREDCODE
653,130794,142600,1426,Yale University,New Haven,CT,06520,New England Association of Schools and College...,www.yale.edu,admissions.yale.edu/yale-net-price-calculator,...,NaN,NaN,NaN,NaN,NaN,5477.0,2.0,4.0,6853.0,NEACHE


In [91]:
partner_domains = [d.strip() for d in open('partner_domains.txt').readlines()]

In [120]:
import re
DOMAIN_RE = re.compile(r'.*[/|.](\w+\.edu).*')

def find_domain(url):
    try:
        return DOMAIN_RE.match(url.lower()).group(1)
    except AttributeError:
        pass

        
df['DOMAIN'] = df['INSTURL'].fillna('').map(find_domain)

In [126]:
partner_data = df[df.DOMAIN.isin(partner_domains)]
excludes = ['Emory University-Oxford College', 'Teachers College at Columbia University']
partner_data = partner_data[~partner_data.INSTNM.isin(excludes)]
partner_unitids = partner_data['UNITID']

In [127]:
partner_data.INSTNM.value_counts().sort_index()

Amherst College                                1
Bowdoin College                                1
Brown University                               1
California Institute of Technology             1
Carleton College                               1
Claremont McKenna College                      1
Colby College                                  1
Colorado College                               1
Columbia University in the City of New York    1
Dartmouth College                              1
Davidson College                               1
Duke University                                1
Emory University                               1
Grinnell College                               1
Hamilton College                               1
Haverford College                              1
Macalester College                             1
Massachusetts Institute of Technology          1
Northwestern University                        1
Oberlin College                                1
Pomona College      

In [128]:
partner_data[['UNITID', 'INSTNM', 'DOMAIN']].to_csv('partner_unitids.csv', index=False)

In [123]:
df[df.UNITID ==198419]

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,D150_L4_BLACKNH,D150_L4_API,D150_L4_AIANOLD,D150_L4_HISPOLD,D_PCTPELL_PCTFLOAN,OPENADMP,UGNONDS,GRADS,ACCREDCODE,DOMAIN
2609,198419,292000,2920,Duke University,Durham,NC,27708,Southern Association of Colleges and Schools C...,WWW.DUKE.EDU,https://npc.collegeboard.org/student/app/duke,...,NaN,NaN,NaN,NaN,6626.0,2.0,138.0,9345.0,SACSCC,duke.edu


In [94]:
# Average net price for $0-$48,000 family income (private for-profit and nonprofit institutions)
partner_data[['UNITID', 'DOMAIN', 'PCTPELL', 'PCTFLOAN', 'PELL_DEBT_MDN']]

,UNITID,DOMAIN,PCTPELL,PCTFLOAN,PELL_DEBT_MDN
223,110404,caltech.edu,0.1292,0.2096,7500
280,112260,cmc.edu,0.1153,0.1514,10644.5
443,121345,pomona.edu,0.1982,0.1467,6850.5
487,123165,scrippscollege.edu,0.1080,0.3313,13000
511,123961,usc.edu,0.2307,0.3569,23250
551,126678,coloradocollege.edu,0.1078,0.3590,19000
651,130697,wesleyan.edu,0.1817,0.3354,14937.5
653,130794,yale.edu,0.1287,0.1039,10026.5
861,139658,emory.edu,0.2032,0.3282,19000
1063,147767,northwestern.edu,0.1376,0.3008,18900


In [100]:
data_old = pd.read_csv('CollegeScorecard_Raw_Data/MERGED2006_07_PP.csv')
partners_old_data = data_old[data_old.UNITID.isin(partner_unitids)]
partners_old_data[['UNITID', 'PCTPELL', 'PCTFLOAN', 'PELL_DEBT_MDN']]

/home/chris/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,6,1408,1431,1432,1433,1475,1489,1537,1538,1539,1540,1542,1606,1610,1611,1614,1615,1616,1729) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,UNITID,PCTPELL,PCTFLOAN,PELL_DEBT_MDN
241,110404,NaN,NaN,PrivacySuppressed
302,112260,NaN,NaN,5925
513,121345,NaN,NaN,PrivacySuppressed
560,123165,NaN,NaN,8341.5
585,123961,NaN,NaN,15250
629,126678,NaN,NaN,11000
741,130697,NaN,NaN,14500
744,130794,NaN,NaN,12783
972,139658,NaN,NaN,14500
1207,147767,NaN,NaN,11000
